In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from matplotlib import pyplot as plt
import pickle
import sys
sys.path.append('./../')
sys.path.append('./../../')
import Portfolio_value as pv
import portfolio_insurance as pi
import scipy

In [2]:
df =  pd.read_csv('./data/snp2.csv')

In [3]:
rf = df['RF'].mean() * 12

In [4]:
p_val = np.load('./result/protection_level/p_val/p_val_0.05.npy', allow_pickle=True)
w_val = np.load('./result/protection_level/w_val/w_val_0.05.npy', allow_pickle=True)

p_val = p_val.reshape(1)
p_val = p_val[0]

w_val = w_val.reshape(1)
w_val = w_val[0]

In [5]:
bear_year = np.load('./result/year/bear_year.npy', allow_pickle=True).reshape(1)[0]


In [6]:
upper_years = [35]

In [7]:
# CPPI_100은 20보다 큰 수에서 좀 찾아보기
# TIPP_80 이하가 되면 좋은데...


In [8]:


# model_names = ['BH_100', 'BH_90', 'BH_80', 'BH_70', 'BH_60', 'BH_50', 'SL_95', 'SP_95', 'CPPI_95_10', 'TIPP_95_40']
# model_names = ['BH_100', 'BH_90', 'BH_80', 'BH_70', 'BH_60', 'BH_50', 'SL_90', 'SP_90', 'CPPI_90_5', 'TIPP_90_30']








In [9]:
bear_perform = {}
for model_name in model_names:
    bear_perform[model_name] = {
    'dep':[],
    'max':[],
    'under':[],
    'inc':[],
    'beq':[],
    'total':[]
}

In [10]:
# PI_Korea랑은 코드가 좀 다름

def get_metric_for_PI(retire_year, k, upper_year, model_name, withrate, const_or_dyna, df, init_wealth, p_val, w_val):
    # bequest
    p_amounts = p_val['{}_rate{}_{}'.format(model_name, withrate, const_or_dyna)][retire_year]
    bequest = p_amounts[12*upper_year]*init_wealth*(1-withrate)/100
    
    # withdraw_amount
    w_amounts = w_val['{}_rate{}_{}'.format(model_name, withrate, const_or_dyna)][retire_year]
    withdraw_amount = sum(w_amounts)
    
    # depletion time
    if np.argwhere(p_amounts == 0).size > 0 : # 고갈이 되면
        dep_time = np.argwhere(p_amounts == 0)[0][0]
    elif np.argwhere(p_amounts == 0).size == 0: # 고갈이 안되면
        dep_time = upper_year*12
        
    # max decline
    dec = np.min(p_amounts)/p_amounts[0]
    dec = dec - 1
    
    # underwater duration
    start_date = datetime(retire_year, 1, 1)
    end_date = start_date + relativedelta(years=upper_year)
    
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m')
    df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    
    under_count = 0
    for i in range(upper_year*12):
        if p_amounts[i]*df.iloc[i]['CPI'] < p_amounts[0]*df.iloc[0]['CPI'] :
             under_count+= 1
        
    return bequest / init_wealth * 100, withdraw_amount / init_wealth * 100, dep_time, dec * 100, under_count

In [11]:
init_wealth = 500000
withdraw_rate = 0.05
k=0
const_or_dyna = 'const'
upper_year = 35

for model_name in model_names:
    print(model_name)
    for years in bear_year[upper_year]:
        for year in years:
            res = get_metric_for_PI(year, k, upper_year, model_name, withdraw_rate, const_or_dyna, df, init_wealth, p_val, w_val)
            if res[2] < upper_year*12 : #고갈 시점이 upper_year보다 작으면 (이미 고갈이 되면)
                bear_perform[model_name]['dep'].append(100)
            elif res[2] >= upper_year*12: # 고갈이 안되면
                bear_perform[model_name]['dep'].append(0)
            bear_perform[model_name]['max'].append(res[3])
            bear_perform[model_name]['under'].append(res[4])
            bear_perform[model_name]['inc'].append(res[1])
            bear_perform[model_name]['beq'].append(res[0])
            bear_perform[model_name]['total'].append(res[1] + res[0])


BH_100
BH_90
BH_80
BH_70
BH_60
BH_50
SL_95
SP_95
CPPI_95_10
TIPP_95_40


In [12]:
avg, std, skew, kurto, SR = {}, {}, {}, {}, {}
ASR, MDD, AvDD, VaR, ES, semi, Omega, Omega2, Omega3 = {}, {}, {}, {}, {}, {}, {}, {}, {}
E, res, ret_sim, ret_sim_y = {}, {}, {}, {}

log_u = {}
quad_u_01 = {}
quad_u_05 = {}
quad_u_10 = {}
power_u_01 = {}
power_u_05 = {}
power_u_09 = {}
exp_u_01 = {}
exp_u_05 = {}
exp_u_10 = {}
exp_u_50 = {}


MPV_1 = {}
MPV_225 = {}
CPV = {}


for items in list(p_val.keys()):
    item = items.split('_')[0] + '_' + items.split('_')[1]
    ret_sim[item], ret_sim_y[item] = [], []
    avg[item], std[item] = [], []
    skew[item], kurto[item] = [], []
    MDD[item], AvDD[item] = [], []
    SR[item] = []
    VaR[item], ES[item], semi[item] = [], [], []
    Omega[item], Omega2[item], Omega3[item] = [], [], []

In [13]:
res_final = {}
for items in list(p_val.keys()):
    item = items.split('_')[0] + '_' + items.split('_')[1]
    res_final[item] = [pd.Series(i[1]) for i in list(p_val[items].items())]


In [14]:

for k in range(len(bear_year[35][0])):    
    for items in list(p_val.keys()):
        item = items.split('_')[0] + '_' + items.split('_')[1]
        res[item] = res_final[item][k]
        ret_m = (res[item] - res[item].shift(1)) / (res[item].shift(1)+0.0000000000000001)
        ret_m = ret_m.iloc[1:]
#         ret_sim[item].extend(list(ret_m*12))
        
        avg_= np.mean(ret_m) * 12
        std_ = np.std(ret_m) * np.sqrt(12)
        avg[item].append(avg_)
        std[item].append(std_)
        skew_ = scipy.stats.skew(ret_m)
        kurto_ = scipy.stats.kurtosis(ret_m)
        skew[item].append(skew_)
        kurto[item].append(kurto_)
        
        SR_ = (avg_ - rf) / (std_+0.0000000000000001)
        SR[item].append(SR_)
        
        
        MDD[item].append(pi.MDD(res[item]))
        AvDD[item].append(pi.AvDD(res[item]))
        
        VaR[item].append(pi.VaR(ret_m, level=5, freq='Y'))
        ES[item].append(pi.ES(ret_m, level=5, freq='Y'))
        semi[item].append(pi.semi(ret_m, freq='Y'))
        
        Omega[item].append(pi.Omega(ret_m, threshold=0, freq='Y'))
        Omega2[item].append(pi.Omega(ret_m, threshold=-0.05, freq='Y'))
        

        
#         ret_sim_y[item].append((res[item].iloc[-1] - res[item].iloc[0])/res[item].iloc[0])
        


In [15]:
for items in list(p_val.keys()):
    item = items.split('_')[0] + '_' + items.split('_')[1]
    ret_sim[item] = pd.Series(ret_sim[item])
    ret_sim_y[item] = pd.Series(ret_sim_y[item])
    
    # 매달 계산한걸 평균
    avg[item] = np.array(avg[item])
    std[item] = np.array(std[item])
    
    skew[item] = np.array(skew[item])#ret_sim[item].skew()
    kurto[item] = np.array(kurto[item])#ret_sim[item].kurtosis()
    SR[item] = np.array(SR[item])#(avg[item].mean() - rf) / (std[item].mean() + 0.0000000000000001)
    
    MDD[item] = np.array(MDD[item])
    AvDD[item] = np.array(AvDD[item])
    
    VaR[item] = np.array(VaR[item])#pi.VaR(ret_sim_y[item], level=5, freq='Y')
    ES[item] = np.array(ES[item])#pi.ES(ret_sim_y[item], level=5, freq='Y')
    semi[item] = np.array(semi[item])#pi.semi(ret_sim_y[item], freq='Y')
    
    Omega[item] = np.array(Omega[item])#pi.Omega(ret_sim[item], threshold=0, freq='Y')
    Omega2[item] = np.array(Omega2[item])#pi.Omega(ret_sim[item], threshold=0, freq='Y')
    



<ipython-input-15-05abcc511f8f>:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  ret_sim[item] = pd.Series(ret_sim[item])
<ipython-input-15-05abcc511f8f>:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  ret_sim_y[item] = pd.Series(ret_sim_y[item])


In [16]:
for items in list(bear_perform.keys()):
    item = items.split('_')[0] + '_' + items.split('_')[1]
    
    bear_perform[items]['avg'] = avg[item]
    bear_perform[items]['std'] = std[item]
    bear_perform[items]['skew'] = skew[item]
    bear_perform[items]['kurto'] = kurto[item]
    bear_perform[items]['SR'] = SR[item]
    bear_perform[items]['MDD'] = MDD[item]
    bear_perform[items]['AvDD'] = AvDD[item]
    bear_perform[items]['VaR'] = VaR[item]
    bear_perform[items]['ES'] = ES[item]
    bear_perform[items]['semi'] = semi[item]
    bear_perform[items]['Omega'] = Omega[item]
    bear_perform[items]['Omega2'] = Omega2[item]
    
    

In [17]:
# Define all dictionaries and indices we want to calculate mean for
dicts = [bear_perform]
indices = model_names

# Use list comprehension to compute all means at once
# Store results in lists
a = [np.round(np.mean(d[i]['dep']),1) for i in indices for d in dicts]
b = [np.round(np.mean(d[i]['max']),1) for i in indices for d in dicts]
c = [np.round(np.mean(d[i]['under']),1) for i in indices for d in dicts]
d = [np.round(np.mean(d[i]['inc']),1) for i in indices for d in dicts]
e = [np.round(np.mean(d[i]['beq']),1) for i in indices for d in dicts]
f = [np.round(np.mean(d[i]['total']),1) for i in indices for d in dicts]

d1 = [np.round(np.mean(d[i]['avg'])*100,2) for i in indices for d in dicts]
d2 = [np.round(np.mean(d[i]['std'])*100,1) for i in indices for d in dicts]
d3 = [np.round(np.mean(d[i]['skew']),2) for i in indices for d in dicts]
d4 = [np.round(np.mean(d[i]['SR']),2) for i in indices for d in dicts]
d5 = [np.round(np.mean(d[i]['MDD'])*100,1) for i in indices for d in dicts]
d6 = [np.round(np.mean(d[i]['AvDD'])*100,1) for i in indices for d in dicts]
d7 = [np.round(np.mean(d[i]['VaR'])*100,1) for i in indices for d in dicts]
d8 = [np.round(np.mean(d[i]['ES'])*100,1) for i in indices for d in dicts]
d9 = [np.round(np.mean(d[i]['semi'])*100,1) for i in indices for d in dicts]
d10 = [np.round(np.mean(d[i]['Omega']),2) for i in indices for d in dicts]
d11 = [np.round(np.mean(d[i]['Omega2']),1) for i in indices for d in dicts]




# If you need to access the results separately, you can do so by indexing the lists:
# For example, a1 corresponds to a_results[0], a2 corresponds to a_results[1], etc.

In [18]:
# print('&& 100/0 & 90/10 & 80/20 & 70/30 & 60/40 & 50/50 & & SL & SP & CPPI & TIPP \\\\ \\hline')
print(model_names)
print('\\textit{Pension-level performance} &&&&&&&&&&&&\\\\')
print('\hspace{3mm} Depletion Prob. (\%) & & ' + ' & '.join(['{}'] * 6).format(*(a[:6])) + ' && ' + ' & '.join(['{}'] * (len(a) - 6)).format(*(a[6:])) + '\\\\')
print('\hspace{3mm} Max. decline (\%) & & ' + ' & '.join(['{}'] * 6).format(*(b[:6])) + ' && ' + ' & '.join(['{}'] * (len(b) - 6)).format(*(b[6:]))+ '\\\\')
print('\hspace{3mm} Underwater duration (month) & & ' + ' & '.join(['{}'] * 6).format(*(c[:6])) + ' && ' + ' & '.join(['{}'] * (len(c) - 6)).format(*(c[6:]))+ '\\\\')
print('\hspace{3mm} Total withdrawal (\%) & & ' + ' & '.join(['{}'] * 6).format(*(d[:6])) + ' && ' + ' & '.join(['{}'] * (len(d) - 6)).format(*(d[6:]))+ '\\\\')
print('\hspace{3mm} Bequest (\%) & & ' + ' & '.join(['{}'] * 6).format(*(e[:6])) + ' && ' + ' & '.join(['{}'] * (len(e) - 6)).format(*(e[6:]))+ '\\\\')
print('\hspace{3mm} Total withdrawal + Bequest (\%) & & ' + ' & '.join(['{}'] * 6).format(*(f[:6])) + ' && ' + ' & '.join(['{}'] * (len(f) - 6)).format(*(f[6:]))+ '\\\\')

print('\\textit{Portfolio-level performance} &&&&&&&&&&&&\\\\')
print('\hspace{3mm} Average return (\%) & & ' + ' & '.join(['{}'] * 6).format(*(d1[:6])) + ' && ' + ' & '.join(['{}'] * (len(d1) - 6)).format(*(d1[6:]))+ '\\\\')
print('\hspace{3mm} Volatility (\%) & & ' + ' & '.join(['{}'] * 6).format(*(d2[:6])) + ' && ' + ' & '.join(['{}'] * (len(d2) - 6)).format(*(d2[6:]))+ '\\\\')
print('\hspace{3mm} Skewness & & ' + ' & '.join(['{}'] * 6).format(*(d3[:6])) + ' && ' + ' & '.join(['{}'] * (len(d3) - 6)).format(*(d3[6:]))+ '\\\\')
print('\hspace{3mm} Sharpe ratio & & ' + ' & '.join(['{}'] * 6).format(*(d4[:6])) + ' && ' + ' & '.join(['{}'] * (len(d4) - 6)).format(*(d4[6:]))+ '\\\\')

print('\\textit{Downside risk} &&&&&&&&&&&&\\\\')
print('\hspace{3mm} MDD (\%) & & ' + ' & '.join(['{}'] * 6).format(*(d5[:6])) + ' && ' + ' & '.join(['{}'] * (len(d5) - 6)).format(*(d5[6:]))+ '\\\\')
print('\hspace{3mm} AvDD (\%) & & ' + ' & '.join(['{}'] * 6).format(*(d6[:6])) + ' && ' + ' & '.join(['{}'] * (len(d6) - 6)).format(*(d6[6:]))+ '\\\\')
print('\hspace{3mm} VaR 5\% (\%) & & ' + ' & '.join(['{}'] * 6).format(*(d7[:6])) + ' && ' + ' & '.join(['{}'] * (len(d7) - 6)).format(*(d7[6:]))+ '\\\\')
print('\hspace{3mm} ES 5\% (\%) & & ' + ' & '.join(['{}'] * 6).format(*(d8[:6])) + ' && ' + ' & '.join(['{}'] * (len(d8) - 6)).format(*(d8[6:]))+ '\\\\')
print('\hspace{3mm} Semideviation (\%) & & ' + ' & '.join(['{}'] * 6).format(*(d9[:6])) + ' && ' + ' & '.join(['{}'] * (len(d9) - 6)).format(*(d9[6:]))+ '\\\\')
print('\hspace{3mm} Omega ratio (0) & & ' + ' & '.join(['{}'] * 6).format(*(d10[:6])) + ' && ' + ' & '.join(['{}'] * (len(d10) - 6)).format(*(d10[6:]))+ '\\\\')
print('\hspace{3mm} Omega ratio (-5\%) & & ' + ' & '.join(['{}'] * 6).format(*(d11[:6])) + ' && ' + ' & '.join(['{}'] * (len(d11) - 6)).format(*(d11[6:]))+ '\\\\')


['BH_100', 'BH_90', 'BH_80', 'BH_70', 'BH_60', 'BH_50', 'SL_95', 'SP_95', 'CPPI_95_10', 'TIPP_95_40']
\textit{Pension-level performance} &&&&&&&&&&&&\\
\hspace{3mm} Depletion Prob. (\%) & & 31.8 & 31.8 & 36.4 & 40.9 & 50.0 & 63.6 && 72.7 & 13.6 & 36.4 & 81.8\\
\hspace{3mm} Max. decline (\%) & & -66.5 & -66.9 & -67.5 & -70.6 & -75.3 & -84.3 && -83.0 & -59.9 & -72.0 & -92.8\\
\hspace{3mm} Underwater duration (month) & & 191.8 & 196.3 & 198.3 & 210.9 & 233.1 & 267.7 && 269.2 & 131.0 & 170.1 & 315.0\\
\hspace{3mm} Total withdrawal (\%) & & 132.0 & 134.2 & 135.5 & 136.1 & 135.8 & 133.3 && 111.1 & 146.1 & 134.6 & 100.5\\
\hspace{3mm} Bequest (\%) & & 199.4 & 146.3 & 102.4 & 66.8 & 39.2 & 19.7 && 48.8 & 122.5 & 46.3 & 8.8\\
\hspace{3mm} Total withdrawal + Bequest (\%) & & 331.4 & 280.5 & 237.9 & 202.9 & 175.1 & 153.0 && 159.9 & 268.6 & 180.8 & 109.3\\
\textit{Portfolio-level performance} &&&&&&&&&&&&\\
\hspace{3mm} Average return (\%) & & -1.35 & -2.17 & -3.27 & -4.42 & -6.29 & -8.05 && -7.35

In [19]:
print('&& 100/0 & 90/10 & 80/20 & 70/30 & 60/40 & 50/50 & & SL & SP & CPPI & TIPP \\\\ \\hline')
print('\hspace{3mm} \\textit{Pension-level performance} &&&&&&&&&&&&\\\\')
print('\hspace{6mm} Depletion Prob. (\%) & & ' + ' & '.join(['{}'] * 6).format(*(a[:6])) + ' && ' + ' & '.join(['{}'] * (len(a) - 6)).format(*(a[6:])) + '\\\\')
print('\hspace{6mm} Max. decline (\%) & & ' + ' & '.join(['{}'] * 6).format(*(b[:6])) + ' && ' + ' & '.join(['{}'] * (len(b) - 6)).format(*(b[6:]))+ '\\\\')
print('\hspace{6mm} Underwater duration (month) & & ' + ' & '.join(['{}'] * 6).format(*(c[:6])) + ' && ' + ' & '.join(['{}'] * (len(c) - 6)).format(*(c[6:]))+ '\\\\')
print('\hspace{6mm} Total withdrawal (\%) & & ' + ' & '.join(['{}'] * 6).format(*(d[:6])) + ' && ' + ' & '.join(['{}'] * (len(d) - 6)).format(*(d[6:]))+ '\\\\')
print('\hspace{6mm} Bequest (\%) & & ' + ' & '.join(['{}'] * 6).format(*(e[:6])) + ' && ' + ' & '.join(['{}'] * (len(e) - 6)).format(*(e[6:]))+ '\\\\')
print('\hspace{6mm} Total withdrawal + Bequest (\%) & & ' + ' & '.join(['{}'] * 6).format(*(f[:6])) + ' && ' + ' & '.join(['{}'] * (len(f) - 6)).format(*(f[6:]))+ '\\\\')

print('\hspace{3mm} \\textit{Portfolio-level performance} &&&&&&&&&&&&\\\\')
print('\hspace{6mm} Average return (\%) & & ' + ' & '.join(['{}'] * 6).format(*(d1[:6])) + ' && ' + ' & '.join(['{}'] * (len(d1) - 6)).format(*(d1[6:]))+ '\\\\')
print('\hspace{6mm} Volatility (\%) & & ' + ' & '.join(['{}'] * 6).format(*(d2[:6])) + ' && ' + ' & '.join(['{}'] * (len(d2) - 6)).format(*(d2[6:]))+ '\\\\')
print('\hspace{6mm} Skewness & & ' + ' & '.join(['{}'] * 6).format(*(d3[:6])) + ' && ' + ' & '.join(['{}'] * (len(d3) - 6)).format(*(d3[6:]))+ '\\\\')
print('\hspace{6mm} Sharpe ratio & & ' + ' & '.join(['{}'] * 6).format(*(d4[:6])) + ' && ' + ' & '.join(['{}'] * (len(d4) - 6)).format(*(d4[6:]))+ '\\\\')

print('\hspace{3mm} \\textit{Downside risk} &&&&&&&&&&&&\\\\')
print('\hspace{6mm} MDD (\%) & & ' + ' & '.join(['{}'] * 6).format(*(d5[:6])) + ' && ' + ' & '.join(['{}'] * (len(d5) - 6)).format(*(d5[6:]))+ '\\\\')
print('\hspace{6mm} AvDD (\%) & & ' + ' & '.join(['{}'] * 6).format(*(d6[:6])) + ' && ' + ' & '.join(['{}'] * (len(d6) - 6)).format(*(d6[6:]))+ '\\\\')
print('\hspace{6mm} VaR 5\% (\%) & & ' + ' & '.join(['{}'] * 6).format(*(d7[:6])) + ' && ' + ' & '.join(['{}'] * (len(d7) - 6)).format(*(d7[6:]))+ '\\\\')
print('\hspace{6mm} ES 5\% (\%) & & ' + ' & '.join(['{}'] * 6).format(*(d8[:6])) + ' && ' + ' & '.join(['{}'] * (len(d8) - 6)).format(*(d8[6:]))+ '\\\\')
print('\hspace{6mm} Semideviation (\%) & & ' + ' & '.join(['{}'] * 6).format(*(d9[:6])) + ' && ' + ' & '.join(['{}'] * (len(d9) - 6)).format(*(d9[6:]))+ '\\\\')
print('\hspace{6mm} Omega ratio (0) & & ' + ' & '.join(['{}'] * 6).format(*(d10[:6])) + ' && ' + ' & '.join(['{}'] * (len(d10) - 6)).format(*(d10[6:]))+ '\\\\')
print('\hspace{6mm} Omega ratio (-5\%) & & ' + ' & '.join(['{}'] * 6).format(*(d11[:6])) + ' && ' + ' & '.join(['{}'] * (len(d11) - 6)).format(*(d11[6:]))+ '\\\\')


&& 100/0 & 90/10 & 80/20 & 70/30 & 60/40 & 50/50 & & SL & SP & CPPI & TIPP \\ \hline
\hspace{3mm} \textit{Pension-level performance} &&&&&&&&&&&&\\
\hspace{6mm} Depletion Prob. (\%) & & 31.8 & 31.8 & 36.4 & 40.9 & 50.0 & 63.6 && 72.7 & 13.6 & 36.4 & 81.8\\
\hspace{6mm} Max. decline (\%) & & -66.5 & -66.9 & -67.5 & -70.6 & -75.3 & -84.3 && -83.0 & -59.9 & -72.0 & -92.8\\
\hspace{6mm} Underwater duration (month) & & 191.8 & 196.3 & 198.3 & 210.9 & 233.1 & 267.7 && 269.2 & 131.0 & 170.1 & 315.0\\
\hspace{6mm} Total withdrawal (\%) & & 132.0 & 134.2 & 135.5 & 136.1 & 135.8 & 133.3 && 111.1 & 146.1 & 134.6 & 100.5\\
\hspace{6mm} Bequest (\%) & & 199.4 & 146.3 & 102.4 & 66.8 & 39.2 & 19.7 && 48.8 & 122.5 & 46.3 & 8.8\\
\hspace{6mm} Total withdrawal + Bequest (\%) & & 331.4 & 280.5 & 237.9 & 202.9 & 175.1 & 153.0 && 159.9 & 268.6 & 180.8 & 109.3\\
\hspace{3mm} \textit{Portfolio-level performance} &&&&&&&&&&&&\\
\hspace{6mm} Average return (\%) & & -1.35 & -2.17 & -3.27 & -4.42 & -6.29 & -8.05